<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>PySpark e Apache Kafka Para Processamento de Dados em Batch e Streaming</font>
## <font color='blue'>Projeto 6</font>
### <font color='blue'>Monitoramento de Criptomoedas em Tempo Real com Kafka, MongoDB e Streamlit</font>
### <font color='blue'>Criação de Visualizações com Dados do MongoDB</font>

In [ ]:
!pip install -q watermark

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kafka
import pymongo
import streamlit
from pymongo import MongoClient
from configparser import RawConfigParser
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Carrega o arquivo de configuração
config_local = RawConfigParser()
config_local.read("dsa_config.conf")

In [ ]:
# Cria o cliente para o MongoDB
client = MongoClient(config_local['MongoDB']['url'])

In [ ]:
# Usa o cliente para conectar ao banco de dados do MongoDB
db = client[config_local['MongoDB']['database']]

In [ ]:
# Acessa a coleção (que é como uma tabela em um banco de dados relacional)
collection = db[config_local['MongoDB']['collection']]

In [ ]:
# Cria a lista de controle
data_list = []

In [ ]:
# Define a query
query = {"timestamp": 1, "data.name": 1, "data.priceUsd": 1, "_id": 0}

In [ ]:
# Executa a query para retornar os documentos armazenados no MongoDB
documents = collection.find({}, query)

In [ ]:
# Loop por cada documento para retornar os dados de interesse
for doc in documents:
    
    # Obtém o timestamp
    timestamp = doc.get("timestamp")
    
    # Se o documento tiver dados
    if "data" in doc:

        # Loop pelos itens
        for data_item in doc["data"]:
          
            # Adiciona os itens à lista de controle
            data_list.append({"timestamp": timestamp, 
                              "name": data_item["name"],
                              "price": round(float(data_item["priceUsd"]), 3)})

In [ ]:
# Converte a lista em dataframe
df = pd.DataFrame(data_list)

In [ ]:
# Visualiza as primeiras linhas
df.head(10)

In [ ]:
# Obtém os nomes únicos das criptomoedas no DataFrame
currencies = df['name'].unique()

# Itera sobre cada criptomoeda
for currency in currencies:
    
    # Filtra o DataFrame para incluir apenas os dados da criptomoeda atual
    df1 = df[df['name'] == currency].copy()
    
    # Converte a coluna 'timestamp' para o formato de datetime
    df1['timestamp'] = pd.to_datetime(df1['timestamp'])
    
    # Formata a coluna 'timestamp' para exibir apenas data e hora no formato desejado
    df1['time'] = df1['timestamp'].dt.strftime('%Y-%m-%d %H:%M')  
    
    # Cria uma nova figura para o gráfico, definindo seu tamanho
    plt.figure(figsize = (20, 6))
    
    # Gera um gráfico de linha para a criptomoeda atual, mostrando o preço ao longo do tempo
    sns.lineplot(data = df1, x = 'time', y = 'price', marker = 'o', label = currency, errorbar = None)
    
    # Adiciona uma legenda com título ao gráfico
    plt.legend(title = "Cotação de Criptomoedas")
    
    # Rotaciona os rótulos do eixo x para melhorar a visualização
    plt.xticks(rotation = 45, ha = 'right')
    
    # Define o rótulo do eixo x como 'Timestamp'
    plt.xlabel('Timestamp')
    
    # Define o rótulo do eixo y como 'Cotação (USD)'
    plt.ylabel('Cotação (USD)')
    
    # Define o título do gráfico
    plt.title('Cotação da Criptomoeda ao Longo do Tempo')
    
    # Exibe o gráfico
    plt.show()

Agora vamos construir uma web app com Streamlit.

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim